# Приоретизация гипотез по увеличению выручки и анализ A/B-теста для крупного интернет-магазина

Цель: приоретизация гипотез и анализ A/B-теста    
Задача: приоретизировать гипотезы , запустить A/B-тест и принять решение: продолжить ли тест и понять насколько он был успешен.

## Описание данных

Файл с гипотезами  /datasets/hypothesis.csv.

`Hypothesis` — краткое описание гипотезы;   
`Reach` — охват пользователей по 10-балльной шкале;     
`Impact` — влияние на пользователей по 10-балльной шкале;    
`Confidence` — уверенность в гипотезе по 10-балльной шкале;    
`Efforts` — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.     

Файл с заказами  /datasets/orders.csv.    

`transactionId` — идентификатор заказа;   
`visitorId` — идентификатор пользователя, совершившего заказ;    
`date` — дата, когда был совершён заказ;    
`revenue` — выручка заказа;    
`group` — группа A/B-теста, в которую попал заказ.    

Файл с пользователями   /datasets/visitors.csv.    

`date` — дата;   
`group` — группа A/B-теста;   
`visitors` — количество пользователей в указанную дату в указанной группе A/B-теста  

## Подготовка данных к работе

подключаем библиотеки для анализа

In [156]:
import pandas as pd
import numpy as np
import scipy.stats as stat
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

# для вывода полного содержимого ячейки
pd.options.display.max_colwidth = 190

грузим исследуемые файлы

In [157]:

hypotesis = pd.read_csv(
    'https://code.s3.yandex.net/datasets/hypothesis.csv')
orders = pd.read_csv('https://code.s3.yandex.net/datasets/orders.csv')
visitors = pd.read_csv('https://code.s3.yandex.net/datasets/visitors.csv')

выводим всю информацию о датафреймах - нужно проверить их на пропуски, дубликаты и прочее

In [158]:
for df, name in zip([hypotesis, orders, visitors],
                    ['hypotesis', 'orders', 'visitors']):
    print(f'\ndataframe: \t{name}\n{"-"*25}'.upper())
    print(df.info())
    display(df.head())

    print(f'\nдубликатов: {df.duplicated().sum()}\n{". "*10}'.upper())

    print(f'\nпропуски\n{". "*10}'.upper())
    print(df.isna().sum())

    print('----'*20)


DATAFRAME: 	HYPOTESIS
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes
None


,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",3,10,8,6
1,"Запустить собственную службу доставки, что сократит срок доставки заказов",2,5,4,10
2,"Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",8,3,7,3
3,"Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей",3,1,1,1



ДУБЛИКАТОВ: 0
. . . . . . . . . . 

ПРОПУСКИ
. . . . . . . . . . 
Hypothesis    0
Reach         0
Impact        0
Confidence    0
Efforts       0
dtype: int64
--------------------------------------------------------------------------------

DATAFRAME: 	ORDERS
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionId  1197 non-null   int64 
 1   visitorId      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB
None


,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B



ДУБЛИКАТОВ: 0
. . . . . . . . . . 

ПРОПУСКИ
. . . . . . . . . . 
transactionId    0
visitorId        0
date             0
revenue          0
group            0
dtype: int64
--------------------------------------------------------------------------------

DATAFRAME: 	VISITORS
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB
None


,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756



ДУБЛИКАТОВ: 0
. . . . . . . . . . 

ПРОПУСКИ
. . . . . . . . . . 
date        0
group       0
visitors    0
dtype: int64
--------------------------------------------------------------------------------


функция форматирования названий столбцов

In [159]:

def get_format_columns(dataframe):
    '''функция принимает датафрейм и возврашает форматированные названия столбцов: 
    (Основное)
    1. без пробелов слева и справа
    2. пробел внутри названия столбца заменяется на символ "_" 
    3. меняет регистр букв на строчный 
    '''

    return list(
        map(
            lambda x: x
            .strip()
            .replace(' ', '_')
            # дополнительное: замена окончаний названий столбцов
            .replace('Id', '_id')
            .lower(),
            dataframe.columns))

форматируем столбцы первым заходом - убираем (даже если их не было) пробелы слев и справа от названия,   


In [160]:
for df, name in zip([hypotesis, orders, visitors],
                    ['hypotesis', 'orders', 'visitors']):


    df.columns = get_format_columns(df)

    print(f'\nDATAFRAME: {name}\n{"-"*10}')

    print(df.columns)


DATAFRAME: hypotesis
----------
Index(['hypothesis', 'reach', 'impact', 'confidence', 'efforts'], dtype='object')

DATAFRAME: orders
----------
Index(['transaction_id', 'visitor_id', 'date', 'revenue', 'group'], dtype='object')

DATAFRAME: visitors
----------
Index(['date', 'group', 'visitors'], dtype='object')


меняем тип даты в датафреймах `orders` и `visitors`

In [161]:
for df, name in zip([orders, visitors],
                    ['orders', 'visitors']):
    
    df['date'] = pd.to_datetime(df.date, format="%Y-%m-%d")

    print(f'\nDATAFRAME: {name}\n{"-"*10}')
    print(df.dtypes)


DATAFRAME: orders
----------
transaction_id             int64
visitor_id                 int64
date              datetime64[ns]
revenue                    int64
group                     object
dtype: object

DATAFRAME: visitors
----------
date        datetime64[ns]
group               object
visitors             int64
dtype: object


данные готовы к работе

## Часть первая. Приоретизация гипотез

Необходимые функции для расчета оценок `ICE` и `RICE`

In [162]:
def get_ice(impact, confidence, efforts):
    '''расчитывает оценку `ICE`   
    1. `impact` - влияние   
    2. `confidence` - уверенность в успехе
    3. `efforts` - обьем затраченных усилий'''

    ice = impact * confidence / efforts
    return round(ice, 2)

In [163]:
def get_rice(reach, ice):
    '''расчитывает оценку `RICE`
    1. `reach` - охват аудитории   '''

    rice = reach * ice
    return round(rice, 4)

рассчитываем `ICE` `RICE`

In [164]:
hypotesis['ice'] = (hypotesis
                    .apply(
                        lambda x:
                            get_ice(x.impact,
                                    x.confidence,
                                    x.efforts),
                            axis=1))


hypotesis['rice'] = (hypotesis
                     .apply(
                         lambda x:
                             get_rice(x.reach,
                                      x.ice),
                             axis=1))

In [165]:
hypotesis[
    ['hypothesis', 'ice']
].sort_values('ice',
              ascending=False)

,hypothesis,ice
8,"Запустить акцию, дающую скидку на товар в день рождения",16.20
0,"Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",13.33
7,"Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",11.20
6,"Показать на главной странице баннеры с актуальными акциями и распродажами, чтобы увеличить конверсию",8.00
2,"Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",7.00
1,"Запустить собственную службу доставки, что сократит срок доставки заказов",2.00
5,"Добавить страницу отзывов клиентов о магазине, что позволит увеличить количество заказов",1.33
3,"Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар",1.12
4,"Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей",1.00


In [166]:
hypotesis.sort_values('rice', ascending=False)

,hypothesis,reach,impact,confidence,efforts,ice,rice
7,"Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",10,7,8,5,11.20,112.00
2,"Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",8,3,7,3,7.00,56.00
6,"Показать на главной странице баннеры с актуальными акциями и распродажами, чтобы увеличить конверсию",5,3,8,3,8.00,40.00
0,"Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",3,10,8,6,13.33,39.99
8,"Запустить акцию, дающую скидку на товар в день рождения",1,9,9,5,16.20,16.20
3,"Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар",8,3,3,8,1.12,8.96
1,"Запустить собственную службу доставки, что сократит срок доставки заказов",2,5,4,10,2.00,4.00
5,"Добавить страницу отзывов клиентов о магазине, что позволит увеличить количество заказов",3,2,2,3,1.33,3.99
4,"Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей",3,1,1,1,1.00,3.00


Самой высокой оценокй RICE обладает гипотеза:    
*Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок*

Оценка RICE является более точной в стравнении с ICE , т.к. затрагивает конкретную часть аудитории для выдвинутой гипотезы.   

## Часть вторая. Анализ А/В-теста

Узнаем сколько проводился тест

In [167]:
print(f"начало теста: {orders['date'].min()}")
print(f"конец теста: {orders['date'].max()}")

начало теста: 2019-08-01 00:00:00
конец теста: 2019-08-31 00:00:00


тест проводился весь август

проверим есть ли пользователи в обеих группах теста

In [168]:
set_visitors_group_a = set(orders.query('group=="A"')['visitor_id'])
set_visitors_group_b = set(orders.query('group=="B"')['visitor_id'])

count_visitors_two_group = len(set_visitors_group_a & set_visitors_group_b)

print('количество пользователей сразу в 2х группах теста:',
      count_visitors_two_group)

количество пользователей сразу в 2х группах теста: 58


посчитаем вероятность

In [169]:
print(
    f"""вероятность попадания одного человека в обе группы:
{
        (count_visitors_two_group / orders['visitor_id']
    .count()*100)
    .round(2)
    } %""")

вероятность попадания одного человека в обе группы:
4.85 %


Отметим тот факт что    
 - группы теста различаются размерами    
  - в обе группы входит 58 человек.   
  
   Удалить их не выйдет так как они находятся в агрегированной таблице с пользователями (`visitors`), а их отсутвие так же исказит результаты

Посчитаем размер групп

In [170]:
group_a = visitors.query('group=="A"')['visitors'].count()
group_b = visitors.query('group=="B"')['visitors'].count()


In [171]:
q = (1 - group_a / group_b) * 100

print(f'разница между группами: {round(q, 2)} %')

разница между группами: 0.0 %


### Кумулятивная выручка по группам

Узнаем кумулятивную выручку по группам теста на протяжении всего теста

Перед этим подготовим данные

создадим таблицу с уникальными парами 'дата-группа теста'

In [172]:
dates_groups = orders[['date', 'group']].drop_duplicates()

агрегируем таблицу с заказами и отсортируем по дате и группе теста по возрастанию

In [173]:
orders_aggregated = (dates_groups
                     .apply(
                         lambda x:
                         orders[
                             np.logical_and(
                                 orders['date'] <= x['date'],

                                 orders['group'] == x['group']
                             )
                         ]

                         .agg(
                             {'date': 'max',

                              'group': 'max',

                              'transaction_id': 'nunique',

                              'visitor_id': 'nunique',

                              'revenue': 'sum'}
                         ),
                         axis=1)
                     .sort_values(
                         ['date', 'group']
                     )
                     )

совершим такую же операцию с таблитцей пользователей

In [174]:
visitors_aggregated = (dates_groups
                       .apply(

                           lambda x:
                               visitors[
                                   np.logical_and(
                                       visitors['date'] <= x['date'],
                                       visitors['group'] == x['group']
                                   )
                               ].agg(
                                   {
                                       'date': 'max',
                                       'group': 'max',
                                       'visitors': 'sum'
                                   }
                                   ), axis=1)
                       .sort_values(
                           ['date', 'group']
                           )
                       )

соберем агрегированные таблицы в одну

In [175]:
cumulative_data = (orders_aggregated
                   .merge(
                       visitors_aggregated,
                       left_on=['date', 'group'],
                       right_on=['date', 'group']
                   )
                   )

In [176]:
cumulative_data.head()

,date,group,transaction_id,visitor_id,revenue,visitors
0,2019-08-01,A,24,20,148579,719
1,2019-08-01,B,21,20,101217,713
2,2019-08-02,A,44,38,242401,1338
3,2019-08-02,B,45,43,266748,1294
4,2019-08-03,A,68,62,354874,1845


In [177]:
cumulative_data.columns

Index(['date', 'group', 'transaction_id', 'visitor_id', 'revenue', 'visitors'], dtype='object')

переименуем столбцы для удобства

In [178]:
(cumulative_data
 .rename(columns={
     'transaction_id': 'orders',
     'visitor_id': 'buyers'},
     inplace=True
 )
)

Строим график кумулятивной выручки по группам теста

In [221]:
fig = px.line(cumulative_data,
              x='date',
              y='revenue',
              color='group',
              labels={
                  'date': 'дата',
                  'revenue': 'кумулятивная выручка',
                  'group': 'группа'}
              )


fig.update_layout(width=800,
                  title='Кумулятивная выручка по группам А/В-теста')
fig.update_traces(line_width=4)

Промежуточный вывод:  
у группы В резко возрастает выручка - это либо дорогой заказ, либо много заказов


### Кумулятивный средний чек по группам

Строим график кумулятивного среднего чека по группам теста

In [180]:
fig = px.line(cumulative_data,
              x='date',
              y=cumulative_data['revenue']/cumulative_data['orders'],
              color='group',
              labels={
                  'date': 'дата',
                  'y': 'кумулятивный ср. чек',
                  'group': 'группа'}
              )

fig.update_layout(width=800,
                  title='Кумулятивный ср. чек по группам А/В-теста')
fig.update_traces(line_width=4)

Промежуточный вывод:  
У группы В наблюдается скачок - значит это был ОЧЕНЬ дорогой заказ

### Относительное изменение кумулятивного среднего чека группы В к группе А

Узнаем относительное изменение кумулятивного среднего чека группы В к группе А 

Разделяем данные по группам теста

In [181]:
cumulative_data_a = cumulative_data[cumulative_data['group'] == 'A']
cumulative_data_b = cumulative_data[cumulative_data['group'] == 'B']

собираем в таблицу но уже подписываем, что куда относится

In [182]:
cumulative_conversion_ab = (cumulative_data_a
                            .merge(
                                cumulative_data_b,
                                how='left',
                                on='date',
                                suffixes=['_a', '_b']
                            )
                            )

находим относительное изменение

In [183]:
k = (
    (cumulative_conversion_ab['revenue_b'] /
     cumulative_conversion_ab['orders_b'])
    /
    (cumulative_conversion_ab['revenue_a'] /
     cumulative_conversion_ab['orders_a']) - 1
)

Строим график

In [184]:

fig = px.line(cumulative_conversion_ab,
              x='date',
              y=k,
              labels={
                  'date': 'дата',
                  'y': 'относительное изменение',
                  'group': 'группа'}
              )

fig.update_layout(width=800,
                  title='Относительное изменение кумулятивного среднего чека группы B к группе А',
                  font_size=10
                  )
fig.add_hline(y=0,
              line_color='red',
              line_dash='dot'
              )
fig.update_traces(line_width=4)

Промежуточный вывод:  
Наблюдаем следущее - средний чек группы В сначала больше, затем снижается до -13%, и резко поднимается до 49% (однозначно находится аномально дорогая покупка) плавно снижаясь до конца месяца

### Кумулятивное ср. количество заказов на посетителя

Вычислим конверсию - среднее количесвтво заказов на поситетиля 

In [227]:
cumulative_data['conversion'] = (cumulative_data['orders'] / 
                                 cumulative_data['visitors'])

Строим график конверсии

In [226]:
fig = px.line(cumulative_data,
              x='date',
              y=cumulative_data['conversion'],
              color='group',
              labels={
                  'date': 'дата',
                  'conversion': 'коверсия',
                  'group': 'группа'})

fig.update_layout(width=800,
                  title='Кумулятивное ср. кол-во заказов на посетителя по группам')

fig.add_hline(y=0.034, line_color='red', line_dash='dot')
fig.add_hline(y=0.03, line_color='blue', line_dash='dot')
fig.update_traces(line_width=3)

Промежуточный вывод:  
Конверсия группы В чуть больше конверсии группы А. Показатели стабилизировались


### Относительное изменение кумулятивного ср. кол-ва заказов на посетителя группы В к группе А

Вычислим относительное изменнеие кумулятивного среденего кол-ва заказов , обозначим его за `k`

In [187]:
k = (
    (cumulative_conversion_ab['orders_b'] /
     cumulative_conversion_ab['visitors_b']
     ) 
    /
    (cumulative_conversion_ab['orders_a'] /
     cumulative_conversion_ab['visitors_a']
     ) - 1
)

Строрим график

In [230]:
fig = px.line(cumulative_conversion_ab,
              x='date',
              y=k*100,
              labels={
                  'date': 'дата',
                  'y': 'относительное изменение, %',
                  'group': 'группа'})


fig.update_layout(
    width=900,
    title='Относительное изменение кумулятивного ср. кол-ва заказов на посетителя группы B к группе А',
    font_size=11)

fig.add_hline(y=0,
              line_color='red',
              line_dash='dot')

fig.update_traces(line_width=4)

Промежуточный вывод:  
Среденне кол-во заказов группы B больше заказов группы А и под конец теста находится между 10% и 15%

### Количество заказов по пользователям

Посчитаем кол-во заказов пользователей чтоб найти аномалии

In [189]:
count_orders = (orders
                .groupby(
                    'visitor_id',
                    as_index=False)
                ['transaction_id']
                .count()
                .rename(
                    columns={
                        'transaction_id': 'count_orders'
                    }
                )
                )



count_orders.head()

,visitor_id,count_orders
0,5114589,1
1,6958315,1
2,8300375,2
3,11685486,1
4,39475350,1


Строим график

In [236]:
fig = px.scatter(count_orders,
                 y='count_orders',
                 labels={
                         'count_orders': 'число заказов',
                         'index': 'номер заказа'
                         }
                 )

fig.update_layout(width=800,
                  title='Количество заказов пользователей')

Видно что пользователи совершали даже 11 покупок. Выясним сколько на самом деле совершают заказов пользователи

найдем 95 и 99 перцентили

In [234]:
bound_count_orders = np.percentile(
    count_orders['count_orders'],
    [95, 99]
)



bound_count_orders

array([2., 4.])

выходит что 95 % пользователей совершают 2 заказа    
а 99 % - менее 4 заказов

Выберем границей нормального числа заказов -  4

In [192]:
bound_normal_orders = bound_count_orders[1]

### Стоимость заказов пользователей

Посмотрим на стоимости заказов пользователей , наверняка есть что-то дорогое

сторим соответсвующий график

In [ ]:
fig = px.scatter(orders,
                 y='revenue',
                 labels={
                   'revenue': 'стоимость',
                     'index': 'заказы'
                 },
                 range_y=[0, 120e3],  # меняем масштаб
                 marginal_y='rug'
                 )

fig.update_layout(
    width=800, title='Стоимость заказов пользователей')

Какая граница стоимостей заказа - найдем 95 и 99 перцентили:

In [194]:
bound_revenue = np.percentile(orders['revenue'],
                              [95, 99]
                              )

bound_revenue

array([28000. , 58233.2])

выходит что 95 и 99 % пользователей совершают покупки до 28_000 и до 58_233.2 соответсвенно.    
Примем, что покупки больше 58_233.2 считаются аномальными

In [195]:
bound_normal_revenue = bound_revenue[1]

### Статистическая значимость различий в среднем кол-ве заказов на посетителя между группами по сырым данным

Сформулируем нулевую и альтернативную гипотезы    

In [196]:
H_0 = "среднее количенство заказов на посетителя обеих групп одинаково"
H_1 = "среднее количенство заказов на посетителя обеих групп разное"

Выберем уровень статистической значимости 5% как общепринятый 

In [197]:
alpha = .05

Проверять гипотезы будем методом Манна-Уитни

разделим заказы по группам

In [198]:
orders_group_a = orders.query('group == "A"')
orders_group_b = orders.query('group == "B"')

разделим заказы по группам но уже посчитаем сколько каждый пользователь совершил покупок

In [199]:
orders_by_user_a = (orders_group_a
                    .groupby(
                        'visitor_id',
                        as_index=False
                    )
                    .agg({'transaction_id': pd.Series.nunique})
                    .rename(
                        columns={'transaction_id': 'count_orders'})
                    )

orders_by_user_b = (orders_group_b
                    .groupby(
                        'visitor_id',
                        as_index=False
                    )
                    .agg({'transaction_id': pd.Series.nunique})
                    .rename(
                        columns={'transaction_id': 'count_orders'}
                    )
                    )

разделим пользователей по группам

In [200]:
visitors_group_a = visitors.query('group == "A"')
visitors_group_b = visitors.query('group == "B"')

запишем для соответсвующи групп пользователей не совершивших покупки

In [201]:
users_group_a_with_zero_orders = (pd.Series(0,
                                            index=np.arange(

                                                visitors_group_a['visitors'].sum() -
                                                len(orders_by_user_a['visitor_id'])
                                            ),
                                            name='count_orders')
                                  )



users_group_b_with_zero_orders = (pd.Series(0,
                                            index=np.arange(
                                                visitors_group_b['visitors'].sum() -
                                                len(orders_by_user_b['visitor_id'])
                                            ),
                                            name='count_orders')
                                  )

соберем покупки пользователей и не совершенные покупки в массивы для соответствующих групп

In [202]:
sample_a = pd.concat(
    [orders_by_user_a['count_orders'],
     users_group_a_with_zero_orders]
)



sample_b = pd.concat(
    [orders_by_user_b['count_orders'],
     users_group_b_with_zero_orders]
)

применим метод Манна-Уитни и рассчитаем относительный прирост ср. числа заказов между группами

In [203]:
p_value = stat.mannwhitneyu(sample_a, sample_b)[1]

prirost = sample_b.mean()/sample_a.mean()-1

напишем функцию которая выведет для наглядности p-value, относительный прирост, и принимаемую гипотезу

In [204]:
def get_info_test(p_value, prirost, info_prirost):
    '''выводит:    
    p-value    
    относительный прирост    
     и принимаемую гипотезу'''

    print("p_value = {:.3f}".format(p_value))
    print(
        "{}: {:.3f}\n".format(info_prirost, prirost))

    if p_value > alpha:
        print("принимаем гипотезу:", H_0)
    else:
        print("принимаем гипотезу:", H_1)

Получаем

In [205]:
get_info_test(p_value, prirost,
              info_prirost="относительный прирост ср. числа заказов группы B по сырым данным")

p_value = 0.017
относительный прирост ср. числа заказов группы B по сырым данным: 0.138

принимаем гипотезу: среднее количенство заказов на посетителя обеих групп разное


### Статистическая значимость различий в среднем чеке заказа  между группами по сырым данным

Сформулируем нулевую и альтернативную гипотезы    

In [206]:
H_0 = "различий в среднем чеке между группами нет"
H_1 = "средние чеки групп разные"

уровень статистической значимости

In [207]:
alpha = 0.05

применим метод Манна-Уитни к средним чекам групп, и рассчитаем относительный прирост

In [208]:
p_value = stat.mannwhitneyu(
    orders_group_a['revenue'],
    orders_group_b['revenue'])[1]

prirost = (
    orders_group_b['revenue'].mean() /
    orders_group_a['revenue'].mean() - 1
)

Получаем

In [209]:
get_info_test(p_value,
              prirost,
              info_prirost="относительный прирост ср. чека заказов группы В по сырым данным")

p_value = 0.729
относительный прирост ср. чека заказов группы В по сырым данным: 0.259

принимаем гипотезу: различий в среднем чеке между группами нет


### Очистка данных от аномалий

Теперь необходимо рассчитать то же самое, но уже по очищенным данным.

План: найти всех аномальных пользователей и убрать их.   
Для начала найдем их в таблице с заказами во всех группах используя границу нормального числа заков найденную ранее при помощи перцентилей

In [210]:
anomal_users_orders = pd.concat(
    [orders_by_user_a.query('count_orders > @bound_normal_orders')['visitor_id'],
     orders_by_user_b.query('count_orders > @bound_normal_orders')['visitor_id']], axis=0)

Теперь отыщем их в таблице с посетителями по границе нормального чека найденного ранее

In [211]:
anomal_users_revenue = orders[
    orders['revenue'] > bound_normal_revenue
]['visitor_id']

соберем их до кучи

In [212]:
anomal_users = (pd.concat(
    [
        anomal_users_orders,
        anomal_users_revenue
    ],
    axis=0)
    .drop_duplicates()
    .sort_values()
)

посмотрим сколько их

In [213]:
print("количество аномальных пользователей: ", anomal_users.shape[0])

количество аномальных пользователей:  15


а теперь прицельно уберем их из групп

In [214]:
orders_by_user_a_without_anomal_users = (orders_by_user_a[
    np.logical_not(
        orders_by_user_a
        .isin(anomal_users)
    )
]
    ['count_orders']
)

orders_by_user_b_without_anomal_users = (orders_by_user_b[
    np.logical_not(
        orders_by_user_b
        .isin(anomal_users)
    )
]
    ['count_orders']
)

и соберем в массивы число заказов и заказов которых не было по группам

In [215]:
sample_a_filtered = pd.concat(
    [
        orders_by_user_a_without_anomal_users,
        users_group_a_with_zero_orders
    ],
    axis=0
)

sample_b_filtered = pd.concat(
    [
        orders_by_user_b_without_anomal_users,
        users_group_b_with_zero_orders
    ],
    axis=0
)

применим метод  Манна-Уитни к очищенным данным

### Статистическая значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным

Сформулируем гипотезы и выберем уровень стат значимости

In [216]:
H_0 = "среднее количенство заказов на посетителя обеих групп одинаково"
H_1 = "среднее количенство заказов на посетителя обеих групп разное"
alpha = .05

применим критерий, найдем относительный прирост 

In [217]:
p_value = stat.mannwhitneyu(
    sample_a_filtered,
    sample_b_filtered)[1]


prirost = sample_b_filtered.mean() / sample_a_filtered.mean() - 1



get_info_test(p_value, 
              prirost,
              info_prirost="прирост среднего количества заказов группы В по очищенным данным")

p_value = 0.017
прирост среднего количества заказов группы В по очищенным данным: 0.138

принимаем гипотезу: среднее количенство заказов на посетителя обеих групп разное


Отметим, что после устранения аномалий p-value и относительный прирост среднего числа закзов не изменился

### Cтатистическая значимость различий в среднем чеке заказа между группами по «очищенным» данным.

Сформулируем гипотезы и уровень стат значимости по прежнему 0,05

In [218]:
H_0 = "средний чек обеих групп одинаковый"
H_1 = "средний чек обеих групп разный"
alpha = .05

перед рассчетами среднего чека  уберем аномальных пользователей

In [219]:
revenue_filter_group_a = (orders[
    np.logical_and(
        orders['group'] == "A",



        np.logical_not(
            orders['visitor_id']
            .isin(anomal_users)
        )
    )
]['revenue']
)



revenue_filter_group_b = (
    orders[
        np.logical_and(
            orders['group'] == "B",
            np.logical_not(
                orders['visitor_id']
                .isin(anomal_users)
            )
        )
    ]['revenue']
)

смотрим результаты

In [220]:
p_value = stat.mannwhitneyu(revenue_filter_group_a, revenue_filter_group_b)[1]
prirost = revenue_filter_group_b.mean() / revenue_filter_group_a.mean() - 1

get_info_test(p_value, prirost,
              info_prirost="прирост среднего чека группы В")

p_value = 0.851
прирост среднего чека группы В: -0.006

принимаем гипотезу: средний чек обеих групп одинаковый


Отметим:    
p-value после устранения аномальных значений увеличился,    
средний чек уменьшился с 26% до -0,6%

## Вывод по результатам A/B-теста

Тест следует завершить и признать неуспешным. Средний чек заказа сегмента В не отличается от сегмета А, и даже уменьшается. 

# Итоговый вывод 

Загрузив необходимые файлы с гипотезами, заказами и посетителями,   
проверив их на пропуски , дубликаты, соответсвие типам данных содержимых столбцов,
приведя названия столбцов к необходимому формату, привели данные к соответствующемк типу данных, 
произвели приоретизацию гипотез методами приоретизации `ICE` и `RICE`.  

Затем вычислили период проведения AB- теста , размер групп и проверили наличие пользователей в обеих руппах и еще проанализировали результаты A/B теста с предварительной подготовкой данных для таких метрик : 
 - кумулятивная вырчка 
- кумулятивный средний чек
- относительное изменение кумулятивного среднего чего группы В к группе А
- кумулятивное среденне количество закзов на посетителя
- относительное изменение кумулятивного среднего количества заказов на посетителя группы В к группе А,   
после чего построили их графики и проанализировали.

Далее подготовили данные для нахождения пределов значений кол-ва заказов и их стоимостей чтоб отсечь аномальные значения

Затем вычислили статистическую значимость различий в :
- среднем количестве заказов на посетителя между группами
- среднем чеке заказа между группами

до и после фильтрации аномалий.


Выяснили следующее:

 -  что приоритетной (по методу `RICE`) гипотезой  является:  

`Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок`.  

  
- тест проводился весь август
- группы теста разделены не равномерно и различаются на 13 %
- что 58 человек находятся сразу в обеих группах , но удалять их не представляется возможным т.к. они агрегированы в таблице с посетителями, по этому было принято оставить их для дальнейшего анализа
- кумулятивная выручка , кумулятивный средний чек, относительное изменение кумулятивного среднего чека группы В к группы А резко возрастает из-за одной большой покупки,
- кумулятивное среднее количество заказов группы В чуть больше чем у группы А и составляет примерно 3,4% против 3%
- относительное изменение кумулятивного среднего кол-ва заказов на посетителя группы В к группе А к концу теста находится в пределах 10-14%
- 99 % пользователей совершают менее 4 покупок
- 99 % пользователей совершают покупки  на сумму менее 58 233.2 рублей

до очистки данных от аномалий 
- прирост ср. количество заказов 13,8% 
- средний чек группы В на 26% больше чем у группы А

после удаления аномальных значений
- ср. кол-во заказов увеличилось до 13,8%
- средний чек группы B уменьшился и сотавил -0.6%.

Т.к. ср. кол-во заказов увеличилось при отсутствии знаяимых изменений по ср. чеку , следовательно стали покупать чаще - выручка увеличилась. 
Поэтому А/В тест  следует завершить и признать для группы В успешным, т.к. поставленная цель по  увеличения выручки выполняется.